In [17]:
import sys
import numpy as np
from tabulate import tabulate

In [18]:
class TripleDict:
    def __init__(self, header=None):
        self._store = []
        self.header = header if header else ["Key", "Value 1", "Value 2"]

    def add(self, key, value1, value2):
        """Adds a new entry, allowing duplicate keys."""
        self._store.append((key, value1, value2))

    def items(self):
        """Returns the stored data as a list of tuples."""
        return self._store

    def print_as_table(self):
        """Prints the stored data in a table format."""
        print(tabulate(self._store, headers=self.header, tablefmt="fancy_grid"))

    def __repr__(self):
        """Represents the TripleDict contents."""
        return f"TripleDict({self._store})"

In [19]:
# Constants
OUTPUT_FILE_PATH = ('My_version.txt')
OUTPUT_ENCODING = 'utf-8'
TABLE_FORMAT = "fancy_grid"
FLOAT_FORMAT = ".2f"


def compute_revenue_values(R, alpha, n, C):
    revenue_values = np.zeros((n, C + 1))
    for i in range(n):
        for x in range(1, C + 1):
            revenue_values[i, x] = R[i] * (1 - np.exp(-alpha[i] * x)) ** x
    return revenue_values

def print_table(data, headers, title):
    """Print table with proper formatting."""
    print(title)
    print(tabulate(data, headers=headers, tablefmt=TABLE_FORMAT, floatfmt=FLOAT_FORMAT))


def solve_resource_allocation(R, alpha, C, n):
    revenue_values = compute_revenue_values(R, alpha, n, C)
    dp = np.zeros((n + 1, C + 1))
    allocation = np.zeros((n + 1, C + 1), dtype=int)

    revenue_table = [[f"i={i + 1}"] + list(revenue_values[i, :]) for i in range(n)]
    headers = ["Prod\\Res"] + [f"c={j}" for j in range(C + 1)]
    print_table(revenue_table, headers, "Revenue Table:")

    data = []
    for i in range(1, n + 1):
        dt = TripleDict([f"Z={i}", f"fi_{i}(z_{i})", f"X_{i}"])
        print(f"\nProcessing product {i}...")
        for c in range(C + 1):
            dp[i, c] = dp[i - 1, c]
            print(f"  Resource count {c}: Initial dp[{i},{c}] = {dp[i, c]}")
            for x in range(1, c + 1):
                print(f"    Calc: dp[{i - 1}, {c - x}] + revenue_values[{i - 1}, {x}]:   {dp[i - 1, c - x]} + {revenue_values[i - 1, x]}")
                value = dp[i - 1, c - x] + revenue_values[i - 1, x]
                print(f"    Trying allocation c={x}, computed value={value}")
                if value > dp[i, c]:
                    print(f"       Value={value} is greater than dp[{i},{c}]={dp[i, c]}")
                    print(f"       Updating dp[{i},{c}] to {value}")
                    print(f"       Updating x to {x}")
                    dp[i, c] = value
                    allocation[i, c] = x
                else:
                    print(f" !!!!           Not updating dp[{i},{c}] = {dp[i, c]}, value = {value}")
            print(f" c= {c}, fi={dp[i, c]}, X={allocation[i, c]}")
            dt.add(c, round(dp[i, c], 2), allocation[i, c])
        data.append(dt)


    # Print DP table for verification
    dp_table = [[f"i={i}"] + list(dp[i, :]) for i in range(1, n + 1)]
    print_table(dp_table, headers, "DP Table:")

    # Print allocation table
    allocation_table = [[f"i={i}"] + list(row[:]) for i, row in enumerate(allocation) if i > 0]
    print_table(allocation_table, headers, "Allocation Table:")

    # Backtrack optimal allocation
    optimal_allocation = np.zeros(n, dtype=int)
    remaining_resources = C
    for i in range(n, 0, -1):
        optimal_allocation[i - 1] = allocation[i, remaining_resources]
        remaining_resources -= optimal_allocation[i - 1]

    # Compute final revenue
    optimal_revenue = dp[n, C]
    return optimal_allocation, optimal_revenue, data
def print_results(optimal_allocation, R, alpha, C, max_revenue):
    """Print the optimal allocation and resultant revenue."""
    print("\nOptimal Resource Allocation:")
    for i, x in enumerate(optimal_allocation):
        revenue = R[i] * (1 - np.exp(-alpha[i] * x)) ** x
        print(f"Customer {i + 1}: {x} resources -> Revenue: {revenue:.2f}")
    print(f"Total resources used: {sum(optimal_allocation)} out of {C}")
    print(f"Maximum revenue: {max_revenue:.2f}")

# Main Execution
R = np.array([5, 4, 10, 8, 3, 7, 2, 8, 3])
alpha = np.array([3, 4, 5.5, 3, 4.5, 2.5, 4, 5, 4.5])
C = 36
n = 9

with open(OUTPUT_FILE_PATH, 'w', encoding=OUTPUT_ENCODING) as output_file:
    sys.stdout = output_file
    optimal_allocation, max_revenue, data = solve_resource_allocation(R, alpha, C, n)
    print_results(optimal_allocation, R, alpha, C, max_revenue)
    sys.stdout = sys.__stdout__

In [21]:
with open("tables_My_Version.txt", 'w', encoding=OUTPUT_ENCODING) as output_file:
    sys.stdout = output_file
    for dt in data:
        dt.print_as_table()
    sys.stdout = sys.__stdout__